# Scrapping de la página de Resoluciones del Ministerio de Salud

<p style='text-align: justify;'>  El código descrito a continuación tiene dos funcionalidades:

1. La primera es traer el número de registros, año e indicativo del año para asi mismo desarrollar el scrapping de la página.
2. La segunda parte, con la ayuda de selenium, hace el scrapping de la página del ministerio simulando los click necesarios para el proceso. 

La finalidad de este script es obtener la ruta de cada uno de los PDFs de las resoluciones del ministerio de salud, publicados en la pagina web y traer esta información en un dataframepara exportar como csv.

En primera medida, se importan las librerias necesarias para el proceso de recolección de datos y llamar la pagina de donda se va a sacar la información preliminar.</p> 

In [ ]:
import pandas as pd
import re
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%pylab inline
url = "https://www.minsalud.gov.co/Paginas/Norm_Resoluciones.aspx"
pagina0 = urlopen(url) 
pagina = bs(pagina0, "lxml")

El siguiente cógido trae el año en una lista en los cuales se han publicado resoluciones.

In [ ]:
año=re.findall('Año</a> : [\w\s]+', str(pagina))
año1=[]
for i in año:    
    año1.append(i.replace("Año</a> : ",""))

El siguiente código trae en una lista el número de publicaciones por cada año, los cuales serán el valor de iteración en el scrapping. Así mismo, se genera el indicativo del año para las iteraciones en la página. 

In [ ]:
lista0=re.findall('<span style="font-weight: lighter;display: inline-block;">\xa0\u200e[\w()]+', str(pagina))
lista1=[]
for i in lista0:    
    lista1.append(i.replace('<span style="font-weight: lighter;display: inline-block;">\xa0\u200e(',"").replace(")",""))
indicativo=[2*i for i in range(1,len(lista1)+1)]

<p style='text-align: justify;'>   El siguiente código genera un DataFrame con el listado de años y el número de resoluciones publicados en la pagina del ministerio de Salud. Se eliminan duplicados, como limpieza de la base de la cual se partira para las iteraciones y el scrapping de las resoluciones de cada año. Finalmente, se seleccionan los años de referencia 2000-2020. </p> 

In [ ]:
base1=pd.DataFrame()
base1['año']=año1
base1['año']=base1['año'].astype(int)
base1['numero_registros']=lista1
base1['numero_registros']=base1['numero_registros'].astype(int)
base1['indicativo']=indicativo
base1=base1.drop_duplicates().reset_index(drop=True)
base1=base1[base1['año']>=2000].reset_index(drop=True)

In [ ]:
base1

<p style='text-align: justify;'>  El siguiente código genera una funcion la cual será usada para extraer cada uno de los links de las resoluciones publicados por MinSalud, esto para posteriormente descargarlos en la máquina local.</p>

<p style='text-align: justify;'>Con ayuda de la librería selenium, webdriver realiza la simulación de en donde abre una venta en Google Chorme (para esto fue necesario descargar la aplicacion "chromedriver.exe"), carga la página en cuestión y selecciona cada uno de los años de publicación segun el indicativo que se defina, posteriormente da un click sobre el link de la resolución que se defina. </p>

<p style='text-align: justify;'>Finalmente,  cuando abre el PDF de cada resoluciones, se trae el link de dicah página y cierra la página de Google Chrome. Cuando el año tienen más de 30 publicaiones, el código da click en página siguiente, según correspondan los casos. </p>

<p style='text-align: justify;'>El código esta diseñado para que cada 30 registros siga a la hoja siguiente dado el año en el que este haciendo la iteración. </p> 

In [ ]:
def busqueda(indicativo,registros):
    browser = webdriver.Chrome(executable_path=r"C:/Users/LENOVO/Desktop/proyecto/chromedriver.exe")
    url = ('https://www.minsalud.gov.co/Paginas/Norm_Resoluciones.aspx')
    browser.get(url)
    browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/a'%indicativo)[0].click()
    time.sleep(15)
    if registros<=30:
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr[%s]/td[3]/a'%(int(indicativo)+1,registros))[0].click()
   
    elif 31<=registros<=60:
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[2]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr[%s]/td[3]/a'%(int(indicativo)+1,registros-30))[0].click()
    
    elif 61<=registros<=90:
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[2]/a'%(int(indicativo)+2))[0].click()
        time.sleep(10)
        try:
            browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        except:
            time.sleep(10)
            browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr[%s]/td[3]/a'%(int(indicativo)+1,registros-60))[0].click()
   
    elif 91<=registros<=120:
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[2]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)        
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr[%s]/td[3]/a'%(int(indicativo)+1,registros-90))[0].click()
   
    elif 121<=registros<=150:
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[2]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5) 
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr[%s]/td[3]/a'%(int(indicativo)+1,registros-120))[0].click()
   
    else:
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[2]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5)
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5) 
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr/td/table/tbody/tr/td[3]/a'%(int(indicativo)+2))[0].click()
        time.sleep(5) 
        browser.find_elements_by_xpath('/html/body/form/div[12]/div[1]/div[2]/div[2]/div[5]/div/div[3]/div[1]/div[2]/div/div[3]/div/div/div/table/tbody/tr/td/table/tbody[%s]/tr[%s]/td[3]/a'%(int(indicativo)+1,registros-150))[0].click()
    pagina=[]
    pagina.append(browser.current_url)
    browser.quit()
    return pagina

<p style='text-align: justify;'>El siguiente código es el encargado de hacer las iteraciones según corresponda el año y el registro a buscar. Esto lo toma del DataFrame generado en el proceso anterior de scrapping donde traía el número de registros publicados en cada año y el indicativo. El resultado es una lista donde se encuentran las rutas de cada resolución. </p>

In [ ]:
links=[]
for i in range(0,base1.shape[0]):
    for j in range(0,base1['numero_registros'][i]):
        print(base1['indicativo'][i], j+1)
        url=busqueda(base1['indicativo'][i],j+1)
        links.append(url[0])

A continuación se muestra un ejemplo con la página del Ministerio de Salud donde se muetra el paso a paso del código anterior:

1. El paso uno, el programa abre la pagina y selecciona el año.
2. En el paso dos, el programa da click en el decreto, según la iteración en la que vaya.
3. En el paso tres, el programa guarda el link de la págian web y cierra la página.
4. Este paso ocurre cuando son más de 30 registros, el programa da click en la siguiente página y vuelve al paso 2.

Paso1: 
<img src="paso_1.JPG" width="800" height="800">

Paso2:
<img src="paso_2.JPG" width="800" height="800">

Paso3:
<img src="paso_3.JPG" width="800" height="800">

Paso4:
<img src="paso_4.JPG" width="800" height="800">

A continuación se transforma la información del scrapping en un dataframe y lo exporta posteriormente en un CSV.

In [ ]:
base2=pd.DataFrame()
base2['links']=links
base2['nombre']=base2['links'].map(lambda x:x.replace("https://www.minsalud.gov.co/Normatividad_Nuevo/","").replace("%20","_"))
base2['años']=base2['nombre'].str.findall(r'[\d]+')
for i in range(len(base2)):
    try:
        base2['años1'][i]=base2['años'][i][-1]
    except:
        base2['años1'][i]=None
base2=base2[['links', 'nombre','años1']]
base2=base2.drop_duplicates().reset_index(drop=True)
base2.to_excel("links_resoluciones.xlsx")